In [84]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic
import pyreadstat
from unidecode import unidecode

In [85]:
#Write a function to do the same that I did with the distance_df data set
def process_distance_data(input_file, output_file):
    """
    Load a CSV file, rename a column for easier merging, and save the processed data as a new CSV file.
    
    Args:
        input_file (str): Path to the input CSV file.
        output_file (str): Path to save the output CSV file.
    """
    distance_df = pd.read_csv(input_file, encoding='latin1')
    distance_df.rename(columns={'country':'mission_country'}, inplace=True)
    distance_df.to_csv(output_file, index=False)

#Apply the function to "C:\Users\HP\Desktop\Felipe\Moises\Distance\Mission_coordinatesok.csv" and save the output to "C:\Users\HP\Desktop\Felipe\Moises\Distance\epp\distance_to_missions_epp.csv"
process_distance_data("C:/Users/HP/Desktop/Felipe/Moises/Distance/Mission_coordinatesok.csv", "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distance_to_missions_epp.csv")

In [86]:
# Define the function to process shapefiles
def shape_to_csv(department_path):
    # Read and reproject the shp files
    shape = gpd.read_file(department_path)
    shape = shape.to_crs(epsg=4326)
    # Calculate the centroid and extract the longitude and latitude
    shape['centroid'] = shape.centroid
    shape['longitude'] = shape['centroid'].x
    shape['latitude'] = shape['centroid'].y
    # Save as CSV in the same folder as the shapefile for easier access
    output_csv = department_path.replace(".shp", ".csv")
    shape.to_csv(output_csv, index=False)

# Root directory containing the folders and shapefiles
root_dir = "C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files"

# Go through all subdirectories and apply the function
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith(".shp") and file.startswith("Distritos_"):
            department_path = os.path.join(root, file) 
            shape_to_csv(department_path)


C:\Users\HP\AppData\Local\Temp\ipykernel_25668\3289591575.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shape['centroid'] = shape.centroid
C:\Users\HP\AppData\Local\Temp\ipykernel_25668\3289591575.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shape['centroid'] = shape.centroid
C:\Users\HP\AppData\Local\Temp\ipykernel_25668\3289591575.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shape['centroid'] = shape.centroid
C:\Users\HP\AppData\Local\Temp\ipykernel_25668\3289591575.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are like

In [87]:
def append_csvs_to_one(output_file, root_dir):
    # List to store individual DataFrames
    all_data = []

    # Walk through all subdirectories to find CSV files
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".csv"):  # Process only CSV files
                file_path = os.path.join(root, file)
                print(f"Appending: {file_path}")
                # Read the CSV and append to the list
                df = pd.read_csv(file_path)
                all_data.append(df)

    # Concatenate all DataFrames into one
    deparmentos_shape_ok = pd.concat(all_data, ignore_index=True)
    # Delete duplicates in CLAVE variable
    deparmentos_shape_ok = deparmentos_shape_ok.drop_duplicates(subset=['CLAVE'])

    # Save the combined DataFrame as a single CSV
    deparmentos_shape_ok.to_csv(output_file, index=False)
    print(f"All CSV files appended and saved to: {output_file}")

#Apply the function
root_dir = "C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files"
output_file = "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distritos_combined.csv"

append_csvs_to_one(output_file, root_dir)

Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\guaira_shp_data.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\alto_parana\Distritos_Alto_Parana.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\caazapa\Distritos_Caazapa.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\guaira\Distritos_Guaira.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\itapua\Distritos_Itapua.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\misiones\Distritos_Misiones.csv
All CSV files appended and saved to: C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distritos_combined.csv


In [88]:

# Function to compute the closest mission for each department
def find_closest_location(distritos, missions):
    # Normalize longitude and latitude, except for rows where DPTO_DESC == 'GUAIRÁ'
    

    results = []

    for index, dep in distritos.iterrows():
        # Skip if department coordinates are invalid
        if pd.isna(dep['latitude']) or pd.isna(dep['longitude']):
            continue

        dep_coords = (dep['latitude'], dep['longitude'])
        
        # Calculate distances to all missions
        missions['distance'] = missions.apply(
            lambda x: geodesic(dep_coords, (x['latitude'], x['longitude'])).kilometers, axis=1
        )
        
        # Find the closest mission and keep the DPTO_DESC from distritos
        closest_mission = missions.loc[missions['distance'].idxmin()]
        results.append({
            'department_index': index,
            'department_code': dep.get('DPTO', 'Unknown'),
            'department_name': dep.get('DPTO_DESC', 'Unknown'),
            'distrito': dep.get('DIST_DESC', 'Unknown'),
            'mission_name': closest_mission['mission_name'],
            'mission_latitude': closest_mission['latitude'],
            'mission_longitude': closest_mission['longitude'],
            'distance_to_mission_km': closest_mission['distance']
        })
    
    return pd.DataFrame(results)

# Example usage
distritos = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distritos_combined.csv")
missions = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distance_to_missions_epp.csv")

# Compute the closest missions
closest_missions_df = find_closest_location(distritos, missions)

# Save the results
closest_missions_df.to_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/closest_missions.csv", index=False)

print("Closest missions have been calculated and saved!")

#Now, make a function so that DIST_DESC observations are in lowercase and clean the name of the cities such that "ascii" characters are removed
def clean_city_names(input_file, output_file):
    # Load the data
    closest_missions = pd.read_csv(input_file, encoding='latin1')
    
    # Lowercase the DIST_DESC column
    closest_missions['distrito'] = closest_missions['distrito'].str.lower()
    # Remove accents and non-ASCII characters
    closest_missions['distrito'] = closest_missions['distrito'].apply(lambda x: unidecode(x))
    # Later, I will merge this data set with the survey data. So, I will use a key with the cleaned city name + department code
    # Thus, If departamento has only one digit, add a leading zero. Also, make department_code string first
    #closest_missions['department_code'] = closest_missions['department_code'].astype(str)
    #closest_missions['department_code'] = closest_missions['department_code'].apply(lambda x: f"0{x}" if len(str(x)) == 1 else x)

    # Save the cleaned data
    closest_missions.to_csv(output_file, index=False, encoding='latin1')
    # Remove non-ASCII characters from the distrito column
   # closest_missions['distrito'] = closest_missions['distrito'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    # Save the cleaned data
    #closest_missions.to_csv(output_file, index=False)
#Apply the function to "C:/Users/HP/Desktop/Felipe/Moises/Distance/closest_missions.csv" and save the output to "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv"
clean_city_names("C:/Users/HP/Desktop/Felipe/Moises/Distance/closest_missions.csv", "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv")

#Now, generate a key to merge closest_missions_cleaned with survey data later.
#The key should be the cleaned city name + department code
#Hence, first geberate a function such that If departamento has only one digit, add a leading zero
def add_leading_zero(x):
    """
    Add a leading zero to single-digit department codes.
    
    Args:
        x (int or str): The department code.
    
    Returns:
        str: The department code with a leading zero if necessary.
    """
    # Convert to string
    x = str(x)
    # Check if the length is 1
    if len(x) == 1:
        return f"0{x}"
    return x

#Now, apply the function to the department_code column of "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv"
closest_missions = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv", encoding='latin1')
closest_missions['department_code'] = closest_missions['department_code'].apply(add_leading_zero)

#Finally, generate the key
closest_missions['key'] = closest_missions['distrito'] + closest_missions['department_code']
#Save the data
closest_missions.to_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv", index=False)

Closest missions have been calculated and saved!


In [89]:
#Define a function to read the survey data
def process_survey_data(input_file, output_file):
    # Read the .dta file using pyreadstat
    survey_data, metadata = pyreadstat.read_dta(input_file)
    # Replace the codes in 'distrito' with their corresponding labels
    distrito_labels = metadata.value_labels[metadata.variable_to_label['distrito']]
    survey_data['distrito'] = survey_data['distrito'].map(distrito_labels)
    # If departamento has only one digit, add a leading zero
    survey_data['departamento'] = survey_data['departamento'].apply(lambda x: f"0{x}" if len(str(x)) == 1 else x)
    # Remove accents and non-ASCII characters
    survey_data['distrito'] = survey_data['distrito'].apply(lambda x: unidecode(x))
    # Save the cleaned data
    survey_data.to_csv(output_file, index=False, encoding='latin1')

#Apply the function to the survey_data
survey_data = process_survey_data("C:/Users/HP/Desktop/Felipe/Encuesta/survey_final.dta", "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/survey_data_cleaned.csv")

In [90]:

#Define a function to generate a key for the survey data so I can merge later with the closest_missions data
def generate_key(survey_data):
    #Create a new column 'key' that combines 'distrito' and dptocode
    survey_data['key'] = survey_data['distrito'] + survey_data['departamento'].astype(str)

#Apply the function to the survey data
survey_data = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/survey_data_cleaned.csv", encoding='latin1')
generate_key(survey_data)
#Save the data
survey_data.to_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/survey_data_cleaned.csv", index=False)

In [91]:
#Now, merge the survey data with the closest_missions data
#First, load the survey data
survey_data = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/survey_data_cleaned.csv", encoding='latin1')
#Load the closest_missions data
closest_missions = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv", encoding='latin1')
#Merge the two data sets
merged_data = pd.merge(survey_data, closest_missions, on='key', how='left')